In [8]:
import sys
sys.path.insert(0, "/Users/kaushik/MyStuff/Workspace/NEU/DS5500/Project/DS5500_CapstoneProject")
import os

In [2]:
from src.config import cfg
from src.data import prepOPPCorpus
from src.data import preprocess

from src.models import CNN
import src.models

from src.main import main
from src.main import driver

from src.utils import gen
from src.utils import metrics

import itertools
import numpy as np
import pandas as pd

# Process

In [ ]:
gen.setSeeds(seed=2021)

In [ ]:
df = gen.loadDataset(cfg)
df

In [ ]:
num_samples = len(df)
num_samples

In [ ]:
df.segment_text = df.segment_text.apply(preprocess.cleanText, lower=True, stem=False)
df.segment_text

In [ ]:
cats = df.category
label_encoder = preprocess.LabelEncoder()
label_encoder.fit(cats)
y = label_encoder.encode(cats)
y

In [ ]:
cats_list = list(itertools.chain.from_iterable(cats.values))
counts = np.bincount([label_encoder.class_to_index[cat_] for cat_ in cats_list])
cat_weights = {i: 1.0 / count for i, count in enumerate(counts)}
cat_weights

In [ ]:
gen.setSeeds(seed=2021)

In [ ]:
train_size = 0.7

In [ ]:
X = df.segment_text.to_numpy()

In [ ]:
X_train, X_, y_train, y_ = preprocess.train_test_split_multilabel(
    X=X, y=y, train_size=train_size
)
X_val, X_test, y_val, y_test = preprocess.train_test_split_multilabel(X=X_, y=y_, train_size=0.5)
test_df = pd.DataFrame({"segment_text": X_test, "category": label_encoder.decode(y_test)})

In [ ]:
print("Successfully split the dataset into {:g}% train, {:g}% val and {:g}% test!".format((train_size)*100, (1-train_size)/2*100, (1-train_size)/2*100))
print("Number of unique segments in total: {}".format(X.shape[0]))
metrics.splitStatistics(splitlist=[X_train, X_val, X_test, y_train, y_val, y_test])

In [ ]:
char_level = False

In [ ]:
tokenizer = preprocess.Tokenizer(char_level=char_level)

tokenizer.fit_on_texts(texts=X_train)

In [ ]:
tokenizer = preprocess.Tokenizer(char_level=char_level)

tokenizer.fit_on_texts(texts=X)

In [ ]:
X_train = np.array(tokenizer.texts_to_sequences(X_train), dtype=object)
X_val = np.array(tokenizer.texts_to_sequences(X_val), dtype=object)
X_test = np.array(tokenizer.texts_to_sequences(X_test), dtype=object)

In [ ]:
c = 0
for lst in X_train:
    if 1 in lst:
        c+=1
c

In [12]:
from gensim.models import FastText, fasttext

In [7]:
from gensim.test.utils import datapath

In [14]:
fpath = os.path.join("../../OPP-115/embeddings/fastText/cc.en.300.bin")

In [15]:
fb_model = fasttext.load_facebook_model(fpath)

In [16]:
fb_model

In [21]:
fb_model.wv['lansgfddu']

array([-0.00759924, -0.01075752, -0.05300984,  0.05575918,  0.0168219 ,
       -0.03344255, -0.0054458 ,  0.00355562,  0.03191954, -0.03460164,
       -0.00831714, -0.01344949,  0.00770235,  0.03292604, -0.03256673,
       -0.05382984, -0.00376443,  0.01920056,  0.02761318, -0.04256339,
       -0.02511978, -0.02395481, -0.0101455 , -0.00598303,  0.03660473,
       -0.02034766, -0.0477565 , -0.00687677, -0.01458233,  0.00946319,
        0.01258066, -0.01718604, -0.03328574, -0.02471119,  0.01493987,
        0.04670187, -0.00043909,  0.0220154 , -0.01299998,  0.00187314,
       -0.00649243, -0.02959694, -0.03681787,  0.03105897, -0.0078527 ,
        0.04084724,  0.00789042, -0.0168241 , -0.02080383, -0.0322049 ,
        0.02371505, -0.02014641,  0.0316869 ,  0.03028911, -0.01438408,
        0.02326871,  0.03058233, -0.01440288, -0.02169228,  0.00351056,
        0.00310671,  0.06041451,  0.04850841,  0.00791368,  0.0061626 ,
       -0.01609514, -0.00237224, -0.00380655, -0.0539527 , -0.02

In [26]:
CU_params = {
    "Data Retention": {
      "precision": 0.8,
      "recall": 0.34782608695652173,
      "f1-score": 0.48484848484848486,
      "support": 23
    },
    "Data Security": {
      "precision": 0.7307692307692307,
      "recall": 0.6666666666666666,
      "f1-score": 0.6972477064220183,
      "support": 57
    },
    "Do Not Track": {
      "precision": 1.0,
      "recall": 1.0,
      "f1-score": 1.0,
      "support": 5
    },
    "First Party Collection/Use": {
      "precision": 0.7107142857142857,
      "recall": 0.8728070175438597,
      "f1-score": 0.7834645669291339,
      "support": 228
    },
    "International and Specific Audiences": {
      "precision": 0.96,
      "recall": 0.9056603773584906,
      "f1-score": 0.9320388349514563,
      "support": 53
    },
    "Introductory/Generic": {
      "precision": 0.6363636363636364,
      "recall": 0.5737704918032787,
      "f1-score": 0.6034482758620691,
      "support": 122
    },
    "Policy Change": {
      "precision": 0.85,
      "recall": 0.5862068965517241,
      "f1-score": 0.6938775510204082,
      "support": 29
    },
    "Practice not covered": {
      "precision": 0.5151515151515151,
      "recall": 0.35051546391752575,
      "f1-score": 0.4171779141104294,
      "support": 97
    },
    "Privacy contact information": {
      "precision": 0.8333333333333334,
      "recall": 0.625,
      "f1-score": 0.7142857142857143,
      "support": 48
    },
    "Third Party Sharing/Collection": {
      "precision": 0.6834862385321101,
      "recall": 0.8370786516853933,
      "f1-score": 0.7525252525252525,
      "support": 178
    },
    "User Access, Edit and Deletion": {
      "precision": 0.8571428571428571,
      "recall": 0.5294117647058824,
      "f1-score": 0.6545454545454545,
      "support": 34
    },
    "User Choice/Control": {
      "precision": 0.6862745098039216,
      "recall": 0.7368421052631579,
      "f1-score": 0.7106598984771574,
      "support": 95
    },
    "micro avg": {
      "precision": 0.7072164948453609,
      "recall": 0.7079463364293086,
      "f1-score": 0.7075812274368232,
      "support": 969
    },
    "macro avg": {
      "precision": 0.7719363005675741,
      "recall": 0.669315460204375,
      "f1-score": 0.7036766378314648,
      "support": 969
    },
    "weighted avg": {
      "precision": 0.7081860164455204,
      "recall": 0.7079463364293086,
      "f1-score": 0.6977651757259035,
      "support": 969
    },
    "samples avg": {
      "precision": 0.7433978873239436,
      "recall": 0.7707453051643192,
      "f1-score": 0.7205860160965795,
      "support": 969
    }
  }

In [27]:
import pandas as pd
pd.DataFrame(CU_params).T

,precision,recall,f1-score,support
Data Retention,0.800000,0.347826,0.484848,23.0
Data Security,0.730769,0.666667,0.697248,57.0
Do Not Track,1.000000,1.000000,1.000000,5.0
First Party Collection/Use,0.710714,0.872807,0.783465,228.0
International and Specific Audiences,0.960000,0.905660,0.932039,53.0
Introductory/Generic,0.636364,0.573770,0.603448,122.0
Policy Change,0.850000,0.586207,0.693878,29.0
Practice not covered,0.515152,0.350515,0.417178,97.0
Privacy contact information,0.833333,0.625000,0.714286,48.0
Third Party Sharing/Collection,0.683486,0.837079,0.752525,178.0


In [28]:
CM_params = {
    "Data Retention": {
      "precision": 1.0,
      "recall": 0.08333333333333333,
      "f1-score": 0.15384615384615385,
      "support": 12
    },
    "Data Security": {
      "precision": 0.9230769230769231,
      "recall": 0.7741935483870968,
      "f1-score": 0.8421052631578947,
      "support": 31
    },
    "Do Not Track": {
      "precision": 1.0,
      "recall": 0.8,
      "f1-score": 0.888888888888889,
      "support": 5
    },
    "First Party Collection/Use": {
      "precision": 0.7835051546391752,
      "recall": 0.8397790055248618,
      "f1-score": 0.8106666666666666,
      "support": 181
    },
    "International and Specific Audiences": {
      "precision": 0.9,
      "recall": 0.8,
      "f1-score": 0.8470588235294118,
      "support": 45
    },
    "Introductory/Generic": {
      "precision": 0.7674418604651163,
      "recall": 0.559322033898305,
      "f1-score": 0.6470588235294118,
      "support": 59
    },
    "Policy Change": {
      "precision": 0.9375,
      "recall": 0.8333333333333334,
      "f1-score": 0.8823529411764706,
      "support": 18
    },
    "Practice not covered": {
      "precision": 0.0,
      "recall": 0.0,
      "f1-score": 0.0,
      "support": 20
    },
    "Privacy contact information": {
      "precision": 0.7916666666666666,
      "recall": 0.6333333333333333,
      "f1-score": 0.7037037037037038,
      "support": 30
    },
    "Third Party Sharing/Collection": {
      "precision": 0.6987179487179487,
      "recall": 0.7676056338028169,
      "f1-score": 0.7315436241610739,
      "support": 142
    },
    "User Access, Edit and Deletion": {
      "precision": 0.8823529411764706,
      "recall": 0.6521739130434783,
      "f1-score": 0.75,
      "support": 23
    },
    "User Choice/Control": {
      "precision": 0.8095238095238095,
      "recall": 0.6296296296296297,
      "f1-score": 0.7083333333333334,
      "support": 54
    },
    "micro avg": {
      "precision": 0.7836879432624113,
      "recall": 0.7129032258064516,
      "f1-score": 0.7466216216216216,
      "support": 620
    },
    "macro avg": {
      "precision": 0.7911487753555093,
      "recall": 0.6143919803571823,
      "f1-score": 0.6637965184994175,
      "support": 620
    },
    "weighted avg": {
      "precision": 0.7694519991158234,
      "recall": 0.7129032258064516,
      "f1-score": 0.7286988781377074,
      "support": 620
    },
    "samples avg": {
      "precision": 0.7210626185958254,
      "recall": 0.7371916508538899,
      "f1-score": 0.7120809614168248,
      "support": 620
    }
  }

In [29]:
pd.DataFrame(CM_params).T

,precision,recall,f1-score,support
Data Retention,1.000000,0.083333,0.153846,12.0
Data Security,0.923077,0.774194,0.842105,31.0
Do Not Track,1.000000,0.800000,0.888889,5.0
First Party Collection/Use,0.783505,0.839779,0.810667,181.0
International and Specific Audiences,0.900000,0.800000,0.847059,45.0
Introductory/Generic,0.767442,0.559322,0.647059,59.0
Policy Change,0.937500,0.833333,0.882353,18.0
Practice not covered,0.000000,0.000000,0.000000,20.0
Privacy contact information,0.791667,0.633333,0.703704,30.0
Third Party Sharing/Collection,0.698718,0.767606,0.731544,142.0


In [32]:
WU_params = {
    "Data Retention": {
      "precision": 1.0,
      "recall": 0.17391304347826086,
      "f1-score": 0.29629629629629634,
      "support": 23
    },
    "Data Security": {
      "precision": 0.8536585365853658,
      "recall": 0.6140350877192983,
      "f1-score": 0.7142857142857142,
      "support": 57
    },
    "Do Not Track": {
      "precision": 1.0,
      "recall": 0.2,
      "f1-score": 0.33333333333333337,
      "support": 5
    },
    "First Party Collection/Use": {
      "precision": 0.8356807511737089,
      "recall": 0.7807017543859649,
      "f1-score": 0.8072562358276644,
      "support": 228
    },
    "International and Specific Audiences": {
      "precision": 1.0,
      "recall": 0.8490566037735849,
      "f1-score": 0.9183673469387755,
      "support": 53
    },
    "Introductory/Generic": {
      "precision": 0.7647058823529411,
      "recall": 0.5327868852459017,
      "f1-score": 0.6280193236714976,
      "support": 122
    },
    "Policy Change": {
      "precision": 0.9333333333333333,
      "recall": 0.4827586206896552,
      "f1-score": 0.6363636363636364,
      "support": 29
    },
    "Practice not covered": {
      "precision": 0.6060606060606061,
      "recall": 0.20618556701030927,
      "f1-score": 0.30769230769230765,
      "support": 97
    },
    "Privacy contact information": {
      "precision": 0.9117647058823529,
      "recall": 0.6458333333333334,
      "f1-score": 0.7560975609756098,
      "support": 48
    },
    "Third Party Sharing/Collection": {
      "precision": 0.8291139240506329,
      "recall": 0.7359550561797753,
      "f1-score": 0.7797619047619048,
      "support": 178
    },
    "User Access, Edit and Deletion": {
      "precision": 0.9375,
      "recall": 0.4411764705882353,
      "f1-score": 0.6,
      "support": 34
    },
    "User Choice/Control": {
      "precision": 0.6987951807228916,
      "recall": 0.6105263157894737,
      "f1-score": 0.651685393258427,
      "support": 95
    },
    "micro avg": {
      "precision": 0.820054945054945,
      "recall": 0.6160990712074303,
      "f1-score": 0.703594578668238,
      "support": 969
    },
    "macro avg": {
      "precision": 0.8642177433468193,
      "recall": 0.5227440615161494,
      "f1-score": 0.6190965877837639,
      "support": 969
    },
    "weighted avg": {
      "precision": 0.8141898219676119,
      "recall": 0.6160990712074303,
      "f1-score": 0.6854934256706405,
      "support": 969
    },
    "samples avg": {
      "precision": 0.7990023474178405,
      "recall": 0.7015845070422535,
      "f1-score": 0.7130365526492287,
      "support": 969
    }
  }

In [33]:
pd.DataFrame(WU_params).T

,precision,recall,f1-score,support
Data Retention,1.000000,0.173913,0.296296,23.0
Data Security,0.853659,0.614035,0.714286,57.0
Do Not Track,1.000000,0.200000,0.333333,5.0
First Party Collection/Use,0.835681,0.780702,0.807256,228.0
International and Specific Audiences,1.000000,0.849057,0.918367,53.0
Introductory/Generic,0.764706,0.532787,0.628019,122.0
Policy Change,0.933333,0.482759,0.636364,29.0
Practice not covered,0.606061,0.206186,0.307692,97.0
Privacy contact information,0.911765,0.645833,0.756098,48.0
Third Party Sharing/Collection,0.829114,0.735955,0.779762,178.0


In [34]:
WM_params = {
    "Data Retention": {
      "precision": 1.0,
      "recall": 0.08333333333333333,
      "f1-score": 0.15384615384615385,
      "support": 12
    },
    "Data Security": {
      "precision": 0.9565217391304348,
      "recall": 0.7096774193548387,
      "f1-score": 0.8148148148148149,
      "support": 31
    },
    "Do Not Track": {
      "precision": 1.0,
      "recall": 0.8,
      "f1-score": 0.888888888888889,
      "support": 5
    },
    "First Party Collection/Use": {
      "precision": 0.8042328042328042,
      "recall": 0.8397790055248618,
      "f1-score": 0.8216216216216216,
      "support": 181
    },
    "International and Specific Audiences": {
      "precision": 0.9705882352941176,
      "recall": 0.7333333333333333,
      "f1-score": 0.8354430379746834,
      "support": 45
    },
    "Introductory/Generic": {
      "precision": 0.8333333333333334,
      "recall": 0.5084745762711864,
      "f1-score": 0.631578947368421,
      "support": 59
    },
    "Policy Change": {
      "precision": 0.8333333333333334,
      "recall": 0.8333333333333334,
      "f1-score": 0.8333333333333334,
      "support": 18
    },
    "Practice not covered": {
      "precision": 1.0,
      "recall": 0.05,
      "f1-score": 0.09523809523809523,
      "support": 20
    },
    "Privacy contact information": {
      "precision": 0.8,
      "recall": 0.5333333333333333,
      "f1-score": 0.64,
      "support": 30
    },
    "Third Party Sharing/Collection": {
      "precision": 0.7878787878787878,
      "recall": 0.7323943661971831,
      "f1-score": 0.7591240875912407,
      "support": 142
    },
    "User Access, Edit and Deletion": {
      "precision": 0.875,
      "recall": 0.30434782608695654,
      "f1-score": 0.4516129032258065,
      "support": 23
    },
    "User Choice/Control": {
      "precision": 0.8611111111111112,
      "recall": 0.5740740740740741,
      "f1-score": 0.6888888888888889,
      "support": 54
    },
    "micro avg": {
      "precision": 0.8286852589641435,
      "recall": 0.6709677419354839,
      "f1-score": 0.7415329768270946,
      "support": 620
    },
    "macro avg": {
      "precision": 0.8934999453594935,
      "recall": 0.5585067167368695,
      "f1-score": 0.634532564399329,
      "support": 620
    },
    "weighted avg": {
      "precision": 0.842847155827211,
      "recall": 0.6709677419354839,
      "f1-score": 0.720337007715936,
      "support": 620
    },
    "samples avg": {
      "precision": 0.7191650853889943,
      "recall": 0.6957621758380771,
      "f1-score": 0.693168880455408,
      "support": 620
    }
  }

In [35]:
pd.DataFrame(WM_params).T

,precision,recall,f1-score,support
Data Retention,1.000000,0.083333,0.153846,12.0
Data Security,0.956522,0.709677,0.814815,31.0
Do Not Track,1.000000,0.800000,0.888889,5.0
First Party Collection/Use,0.804233,0.839779,0.821622,181.0
International and Specific Audiences,0.970588,0.733333,0.835443,45.0
Introductory/Generic,0.833333,0.508475,0.631579,59.0
Policy Change,0.833333,0.833333,0.833333,18.0
Practice not covered,1.000000,0.050000,0.095238,20.0
Privacy contact information,0.800000,0.533333,0.640000,30.0
Third Party Sharing/Collection,0.787879,0.732394,0.759124,142.0
